In [1]:
#Install Required Packages
!pip install requests
!pip install pandas


In [2]:
#Run re required packages
import requests
import pandas as pd

# BUS DELAY

In [3]:
# Toronto Open Data is stored in a CKAN instance. Its APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# To hit our API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources".
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = {"id": "ttc-bus-delay-data"}
package = requests.get(url, params=params).json()

# Create an empty DataFrame to store the data
final_df = pd.DataFrame()

# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

    # To get metadata for non datastore_active resources:
    if not resource["datastore_active"]:
        url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
        resource_metadata = requests.get(url).json()

        # From here, you can use the "url" attribute to download this file
        download_url = resource_metadata["result"]["url"]

        # Exclude the specified file
        if download_url == "https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/e271cdae-8788-4980-96ce-6a5c95bc6618/resource/71bb1283-7388-4d23-aa4e-0f393a80abce/download/ttc-bus-delay-data-readme.xlsx":
            continue

        # Download the data from the resource
        try:
            dfs = pd.read_excel(download_url, sheet_name=None)
            # Concatenate all DataFrames from different sheets into a single DataFrame
            for sheet_name, df in dfs.items():
                # Append the new data to the final DataFrame
                final_df = pd.concat([final_df, df], ignore_index=True)
        except Exception as e:
            print(f"Error fetching data from {download_url}: {e}")
            continue

# Get the count of new data rows
new_data_count = len(final_df)

#Since data was concatenated, some columns didn't match. We fix them here
final_df['Date'] = final_df['Date'].fillna(final_df['Report Date'])
final_df['Line'] = final_df['Line'].fillna(final_df['Route'])
final_df['Delay'] = final_df['Delay'].fillna(final_df['Min Delay'].combine_first(final_df[' Min Delay']))
final_df['Gap'] = final_df['Gap'].fillna(final_df['Min Gap'])
final_df['Bound'] = final_df['Bound'].fillna(final_df['Direction'])

#Convert to String to Title case to correct how some values were captures
final_df['Location'] = final_df['Location'].astype(str).apply(lambda x: x.title())

# Generate location to include City, Province, and Country - Needed to generate map coordinates
final_df['Location Full'] = final_df['Location'] + ", Toronto, ON, CA"

# Drop the 'Report Date' column
final_df.drop(columns=['Report Date', 'Route', 'Min Delay', 'Min Gap', ' Min Delay', 'Direction', 'Incident ID', 'Unnamed: 10'], inplace=True)

# Save the final DataFrame as a CSV file
final_df.to_csv("ttc_bus_delay_data.csv", index=False)


print("Count of new data:", new_data_count)

Count of new data: 611788


# STREET CAR DELAY

In [4]:
# Toronto Open Data is stored in a CKAN instance. Its APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# To hit our API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources".
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = {"id": "ttc-streetcar-delay-data"}
package = requests.get(url, params=params).json()

# Create an empty DataFrame to store the data
final_df = pd.DataFrame()

# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

    # To get metadata for non datastore_active resources:
    if not resource["datastore_active"]:
        url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
        resource_metadata = requests.get(url).json()

        # From here, you can use the "url" attribute to download this file
        download_url = resource_metadata["result"]["url"]

        # Exclude the specified file
        if download_url == "https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/b68cb71b-44a7-4394-97e2-5d2f41462a5d/resource/0fe61851-c67b-49bc-8c27-3a89b33b43af/download/ttc-streetcar-delay-data-readme.xlsx":
            continue

        # Download the data from the resource
        try:
            dfs = pd.read_excel(download_url, sheet_name=None)

            # Concatenate all DataFrames from different sheets into a single DataFrame
            for sheet_name, df in dfs.items():

                # Append the new data to the final DataFrame
                final_df = pd.concat([final_df, df], ignore_index=True)
        except Exception as e:
            print(f"Error fetching data from {download_url}: {e}")
            continue

# Get the count of new data rows
new_data_count = len(final_df)

# Fill up null Date fields with Report Date
final_df['Date'] = final_df.apply(lambda row: row['Date'] if pd.notnull(row['Date']) else row['Report Date'], axis=1)
final_df['Line'] = final_df.apply(lambda row: row['Line'] if pd.notnull(row['Line']) else row['Route'], axis=1)
final_df['Delay'] = final_df.apply(lambda row: row['Delay'] if pd.notnull(row['Delay']) else row['Min Delay'], axis=1)
final_df['Gap'] = final_df.apply(lambda row: row['Gap'] if pd.notnull(row['Gap']) else row['Min Gap'], axis=1)
final_df['Bound'] = final_df.apply(lambda row: row['Bound'] if pd.notnull(row['Bound']) else row['Direction'], axis=1)

#Convert column to to String and then Title case to correct how some values were captures
final_df['Location'] = final_df['Location'].astype(str).apply(lambda x: x.title())

# Generate location to include City, Province, and Country - Needed to generate map coordinates
final_df['Location Full'] = final_df['Location'] + ", Toronto, ON, CA"

# Drop the 'Report Date' column
final_df.drop(columns=['Report Date', 'Route', 'Min Delay', 'Min Gap', 'Direction', 'Incident ID'], inplace=True)

# Save the final DataFrame as a CSV file
final_df.to_csv("ttc_streetcar_delay_data.csv", index=False)

print("Count of new data:", new_data_count)


Count of new data: 132280


# SUBWAY DELAY

In [5]:
# Toronto Open Data is stored in a CKAN instance. Its APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# To hit our API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources".
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = {"id": "ttc-subway-delay-data"}
package = requests.get(url, params=params).json()

# Create an empty DataFrame to store the data
final_df = pd.DataFrame()

# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

    # To get metadata for non datastore_active resources:
    if not resource["datastore_active"]:
        url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
        resource_metadata = requests.get(url).json()

        # From here, you can use the "url" attribute to download this file
        download_url = resource_metadata["result"]["url"]

        # Exclude the specified file
        if download_url == "https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/996cfe8d-fb35-40ce-b569-698d51fc683b/resource/3900e649-f31e-4b79-9f20-4731bbfd94f7/download/ttc-subway-delay-codes.xlsx":
            continue
        if download_url == "https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/996cfe8d-fb35-40ce-b569-698d51fc683b/resource/ca43ac3d-3940-4315-889b-a9375e7b8aa4/download/ttc-subway-delay-data-readme.xlsx":
            continue
        
        # Download the data from the resource

        # Download the data from the resource
        try:
            dfs = pd.read_excel(download_url, sheet_name=None)

            # Concatenate all DataFrames from different sheets into a single DataFrame
            for sheet_name, df in dfs.items():

                # Append the new data to the final DataFrame
                final_df = pd.concat([final_df, df], ignore_index=True)
        except Exception as e:
            print(f"Error fetching data from {download_url}: {e}")
            continue

# Get the count of new data rows
new_data_count = len(final_df)

# Delete the first 3 columns in the final DataFrame - NOT RELEVANT TO ME
final_df = final_df.drop(columns=final_df.columns[:3])

# Delete rows 2 to 10 in the final DataFrame - NOT RELEVANT TO ME.
final_df = final_df.drop(final_df.index[0:10])

#Convert to String to Title case to correct how some values were captures
final_df['Location'] = final_df['Station'].astype(str).apply(lambda x: x.title())

# Generate location to include City, Province, and Country - Needed to generate map coordinates
final_df['Location Full'] = final_df['Location'] + ", Toronto, ON, CA"

# Drop the 'Station' column
final_df.drop(columns=['Station'], inplace=True)


# Save the final DataFrame as a CSV file
final_df.to_csv("ttc_subway_delay_data.csv", index=False)

print("Count of new data:", new_data_count)

Count of new data: 184485
